# Segmenting and Clustering Neighborhoods in Toronto

First we install Beautiful Soup 4 and the lxml parser:

In [3]:
!conda install -c conda-forge beautifulsoup4 -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.7.1       |        py36_1001         140 KB  conda-forge
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_0         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.0 MB

The following packages will be UPDATED:

  beautifulsoup4      anaconda::beautifulsoup4-4.7.1-py36_1 --> conda-forge::beautifulsoup4-4.7.1-py36_1001
  conda                                        4.6.4-py36_0 --> 4.6.7-py36_0
  openssl                              1.1.1a-h14c3975_1000 --> 1.1.1b-h14c3975_0



beautifulsoup4-4.7.1 | 140 KB    | ##

In [5]:
!conda install -c conda-forge lxml -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    lxml-4.3.2                 |   py36h23eabaa_0         1.5 MB  conda-forge
    openssl-1.1.1b             |       h14c3975_0         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.3 MB

The following packages will be UPDATED:

  conda                                        4.6.4-py36_0 --> 4.6.7-py36_0
  lxml                  anaconda::lxml-4.3.0-py36hefd8a0e_0 --> conda-forge::lxml-4.3.2-py36h23eabaa_0
  openssl                              1.1.1a-h14c3975_1000 --> 1.1.1b-h14c3975_0



lxml-4.3.2           | 1.5 MB    | #################

Next import necessary packages:

In [24]:
from bs4 import BeautifulSoup as bs
import requests
import csv
import pandas as pd

In [25]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #fetches the webpage as a text file

In the following cell we scrape the webpage into a csv file with the necessary parameters:

In [26]:
soup = bs(source, 'lxml') 
table = soup.find('table') # finds the table to exclude other unwanted html
csv_file = open('tps_scrape.csv', 'w') #creates a csv file
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood']) #name of the columns in our csv file
for row in table.find_all('tr'): #this loops through each row in the table
    prevro = ro
    ro = []
    for entry in row.find_all('td'): #loops through each entry in the row
        ro.append(entry.text)
    if len(ro) and len(prevro) != 0: # this excludes the first list which has length zero
        new = ro[2].replace("\n", "") #fixes the format
        ro[2] = new
        if ro[2] == 'Not assigned': # assigns the borough as the neighborhood for those with 'not assigned' neighborhood
            ro[2] = ro[1]
        if prevro[0] == ro[0]: # checks if the previous row is identical to the current row then combines the neighborhoods if they are 
            ro = [ro[0], ro[1], prevro[2] + ', ' + ro[2]]
        if ro[1] and prevro[1] != 'Not assigned': #this logic excludes unaasigned postcodes and writes the previous row to a csv file
            if ro[0] != prevro[0]:
                csv_writer.writerow(prevro) #note that this code would not print the last line of the table, but in this case we don't need to fix that since the last postcode is unassigned
csv_file.close()

Assign the data to a Pandas DataFrame and find the shape:

In [32]:
tpc_df = pd.read_csv('tps_scrape.csv')
tpc_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [31]:
tpc_df.shape

(103, 3)

In [47]:
df1 = pd.read_csv('Geospatial_Coordinates.csv')
df1.rename(index = str, columns ={'Postal Code': 'Postcode'}, inplace=True )
df = df1.merge(tpc_df, how= 'inner', on = 'Postcode')
df

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,43.711112,-79.284577,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"


In [48]:
df.head()

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
